## **Datos Vivienda**

In [27]:
import pandas as pd
import numpy as np
import os
import sys

In [28]:
municipios_cluster = pd.read_csv("external_data\municipios_con_cluster.csv", sep=",")

### **superficieMediaFamiliares**

In [29]:
import os
import re
import unicodedata
import pandas as pd

# -----------------------------
# CONFIG: rutas de entrada
# -----------------------------
POBLACION_CSV = "external_data/poblacion_total.csv"          # sep=";"
SUPERFICIE_CSV = "data/vivienda/superficie_vivienda.csv"    # sep=";"  <-- ajusta ruta/nombre

# -----------------------------
# Helpers
# -----------------------------
def parse_es_number(x):
    """Convierte '4.868' -> 4868, '26.226,00' -> 26226.0, etc."""
    if pd.isna(x):
        return None
    s = str(x).strip()
    if s == "" or s.lower() == "nan":
        return None
    s = s.replace(".", "").replace(",", ".")
    try:
        return float(s)
    except ValueError:
        return None

def normalize_nombre(nombre):
    """
    - Quita código INE al inicio: '28001 Acebeda, La' -> 'Acebeda, La'
    - Convierte 'X, La/El/Los/Las' -> 'X (La/El/Los/Las)'
    """
    if pd.isna(nombre):
        return nombre
    s = str(nombre).strip()
    s = re.sub(r"^\d+\s+", "", s)  # quita INE

    if ", " in s and "(" not in s and ")" not in s:
        parts = s.split(", ")
        if len(parts) == 2 and parts[1] in {"La", "El", "Los", "Las"}:
            s = f"{parts[0].strip()} ({parts[1].strip()})"

    s = re.sub(r"\s+", " ", s)
    return s

def canon_key(nombre):
    """Clave canónica para casar nombres aunque haya tildes, comas, espacios, etc."""
    s = normalize_nombre(nombre)
    if pd.isna(s):
        return s
    s = unicodedata.normalize("NFKD", s)
    s = "".join(c for c in s if not unicodedata.combining(c))  # quita tildes
    s = s.lower()
    s = re.sub(r"[^a-z0-9]+", "", s)  # deja solo alfanumérico
    return s

# ============================================================
# 1) CARGA Y PREPARACIÓN POBLACIÓN (largo -> ancho)
# ============================================================
pobl = pd.read_csv(POBLACION_CSV, sep=";", dtype=str)

pobl["Sexo"] = pobl["Sexo"].astype(str).str.strip()
pobl = pobl[pobl["Sexo"] == "Total"].copy()

pobl["Nombre"] = pobl["Municipios"].apply(normalize_nombre)
pobl["key"] = pobl["Nombre"].apply(canon_key)
pobl["Periodo"] = pd.to_numeric(pobl["Periodo"], errors="coerce").astype("Int64")
pobl["Total"] = pobl["Total"].apply(parse_es_number)

total_poblacion = (
    pobl.pivot_table(index=["key", "Nombre"], columns="Periodo", values="Total", aggfunc="first")
    .reset_index()
)
total_poblacion.columns = ["key", "Nombre"] + [str(c) for c in total_poblacion.columns[2:]]

# ============================================================
# 2) CARGA Y PREPARACIÓN SUPERFICIE (ya ancho)
# ============================================================
sup = pd.read_csv(SUPERFICIE_CSV, sep=";", dtype=str)

sup["Nombre"] = sup["Nombre"].apply(normalize_nombre)
sup["key"] = sup["Nombre"].apply(canon_key)

year_cols = [c for c in sup.columns if re.fullmatch(r"\d{4}", str(c))]
for c in year_cols:
    sup[c] = sup[c].apply(parse_es_number)

superficie_total = sup[["key", "Nombre"] + year_cols].copy()

# ============================================================
# 3) PREPARAR CLUSTERS (municipios_cluster ya cargado)
# ============================================================
municipios_cluster = municipios_cluster.copy()
municipios_cluster["Nombre"] = municipios_cluster["Nombre"].apply(normalize_nombre)
municipios_cluster["key"] = municipios_cluster["Nombre"].apply(canon_key)

municipios_cluster_ren = municipios_cluster.rename(columns={"Cluster": "cluster"})
municipios_cluster_ren["cluster"] = pd.to_numeric(municipios_cluster_ren["cluster"], errors="coerce")

# ============================================================
# 4) AÑOS A PROCESAR (intersección población vs superficie)
# ============================================================
years_pobl = {int(c) for c in total_poblacion.columns if re.fullmatch(r"\d{4}", str(c))}
years_sup = {int(c) for c in year_cols}

years_to_process = sorted(years_pobl.intersection(years_sup))
print("Años a procesar:", years_to_process)

# ============================================================
# 5) FUNCIÓN PRINCIPAL
# ============================================================
def procesar_anio(anio: int):
    col = str(int(anio))

    numero_superficie = (
        superficie_total[["key", "Nombre", col]]
        .rename(columns={col: "superficie_total"})
    )

    poblacion_anio = (
        total_poblacion[["key", col]]
        .rename(columns={col: "total_poblacion"})
    )

    datos = pd.merge(numero_superficie, poblacion_anio, on="key", how="inner")

    datos = pd.merge(
        datos,
        municipios_cluster_ren[["key", "Nombre", "cluster"]],
        on="key",
        how="left",
        suffixes=("", "_cluster")
    )

    datos["Nombre"] = datos["Nombre_cluster"].fillna(datos["Nombre"])
    datos = datos.drop(columns=["Nombre_cluster"])

    datos["superficie_total"] = pd.to_numeric(datos["superficie_total"], errors="coerce")
    datos["total_poblacion"] = pd.to_numeric(datos["total_poblacion"], errors="coerce")

    # Aquí no es ratio: el indicador es directamente la superficie
    datos["superficieFamiliares"] = datos["superficie_total"].fillna(0)

    datos["type"] = 1  # más es mejor

    def calcular_atractividad_por_grupo(grupo: pd.DataFrame) -> pd.DataFrame:
        grupo = grupo.copy()
        media = grupo["superficieFamiliares"].mean()
        desviacion = grupo["superficieFamiliares"] - media
        max_dev = desviacion.abs().max()

        grupo["atractividad"] = (desviacion / max_dev) if (max_dev and max_dev != 0) else 0

        grupo["atractividad_0_100"] = grupo.apply(
            lambda row: 100 * ((row["atractividad"] + 1) / 2) if row["type"] == 1
            else 100 - (100 * ((row["atractividad"] + 1) / 2)),
            axis=1
        ).round(2)

        return grupo

    con_cluster = datos[datos["cluster"].notna()].copy()
    sin_cluster = datos[datos["cluster"].isna()].copy()

    con_cluster = con_cluster.groupby("cluster", group_keys=False).apply(calcular_atractividad_por_grupo)

    if not sin_cluster.empty:
        sin_cluster["atractividad_0_100"] = 0.0

    datos_final = pd.concat([con_cluster, sin_cluster], ignore_index=True)

    resultado = datos_final[["Nombre", "atractividad_0_100"]].rename(
        columns={"atractividad_0_100": "superficieFamiliares"}
    )

    output_path = f"data_interfaz/vivienda/normalizacion_final/{anio}/"
    os.makedirs(output_path, exist_ok=True)
    resultado.to_csv(f"{output_path}superficieFamiliares.csv", index=False)

    print(f"Archivo del {anio} exportado a {output_path}")

# ============================================================
# 6) PROCESAR (si solo quieres 2021, deja esto)
# ============================================================
# procesar_anio(2021)

# Si quisieras todos:
for y in years_to_process:
    procesar_anio(y)


Años a procesar: [2024]
Archivo del 2024 exportado a data_interfaz/vivienda/normalizacion_final/2024/


C:\Users\aitan\AppData\Local\Temp\ipykernel_17672\3814085667.py:165: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  con_cluster = con_cluster.groupby("cluster", group_keys=False).apply(calcular_atractividad_por_grupo)


### **Familiares**

In [30]:
import os
import re
import unicodedata
import pandas as pd

# -----------------------------
# CONFIG: rutas de entrada
# -----------------------------
POBLACION_CSV = "external_data/poblacion_total.csv"          # sep=";"
FAMILIARES_CSV = "data/vivienda/viviendas_familiares.csv"     # sep=";"  <-- ajusta ruta/nombre

# -----------------------------
# Helpers
# -----------------------------
def parse_es_number(x):
    """Convierte '4.868' -> 4868, '26.226,00' -> 26226.0, etc."""
    if pd.isna(x):
        return None
    s = str(x).strip()
    if s == "" or s.lower() == "nan":
        return None
    s = s.replace(".", "").replace(",", ".")
    try:
        return float(s)
    except ValueError:
        return None

def normalize_nombre(nombre):
    """
    - Quita código INE al inicio: '28001 Acebeda, La' -> 'Acebeda, La'
    - Convierte 'X, La/El/Los/Las' -> 'X (La/El/Los/Las)'
    """
    if pd.isna(nombre):
        return nombre
    s = str(nombre).strip()
    s = re.sub(r"^\d+\s+", "", s)  # quita INE

    if ", " in s and "(" not in s and ")" not in s:
        parts = s.split(", ")
        if len(parts) == 2 and parts[1] in {"La", "El", "Los", "Las"}:
            s = f"{parts[0].strip()} ({parts[1].strip()})"

    s = re.sub(r"\s+", " ", s)
    return s

def canon_key(nombre):
    """Clave canónica para casar nombres aunque haya tildes, comas, espacios, etc."""
    s = normalize_nombre(nombre)
    if pd.isna(s):
        return s
    s = unicodedata.normalize("NFKD", s)
    s = "".join(c for c in s if not unicodedata.combining(c))  # quita tildes
    s = s.lower()
    s = re.sub(r"[^a-z0-9]+", "", s)  # deja solo alfanumérico
    return s

# ============================================================
# 1) CARGA Y PREPARACIÓN POBLACIÓN (largo -> ancho)
# ============================================================
pobl = pd.read_csv(POBLACION_CSV, sep=";", dtype=str)

pobl["Sexo"] = pobl["Sexo"].astype(str).str.strip()
pobl = pobl[pobl["Sexo"] == "Total"].copy()

pobl["Nombre"] = pobl["Municipios"].apply(normalize_nombre)
pobl["key"] = pobl["Nombre"].apply(canon_key)
pobl["Periodo"] = pd.to_numeric(pobl["Periodo"], errors="coerce").astype("Int64")
pobl["Total"] = pobl["Total"].apply(parse_es_number)

total_poblacion = (
    pobl.pivot_table(index=["key", "Nombre"], columns="Periodo", values="Total", aggfunc="first")
    .reset_index()
)
total_poblacion.columns = ["key", "Nombre"] + [str(c) for c in total_poblacion.columns[2:]]

# ============================================================
# 2) CARGA Y PREPARACIÓN FAMILIARES (ya ancho)
# ============================================================
fam = pd.read_csv(FAMILIARES_CSV, sep=";", dtype=str)

fam["Nombre"] = fam["Nombre"].apply(normalize_nombre)
fam["key"] = fam["Nombre"].apply(canon_key)

year_cols = [c for c in fam.columns if re.fullmatch(r"\d{4}", str(c))]
for c in year_cols:
    fam[c] = fam[c].apply(parse_es_number)

familiares_total = fam[["key", "Nombre"] + year_cols].copy()

# ============================================================
# 3) PREPARAR CLUSTERS (municipios_cluster ya cargado)
# ============================================================
municipios_cluster = municipios_cluster.copy()
municipios_cluster["Nombre"] = municipios_cluster["Nombre"].apply(normalize_nombre)
municipios_cluster["key"] = municipios_cluster["Nombre"].apply(canon_key)

municipios_cluster_ren = municipios_cluster.rename(columns={"Cluster": "cluster"})
municipios_cluster_ren["cluster"] = pd.to_numeric(municipios_cluster_ren["cluster"], errors="coerce")

# ============================================================
# 4) AÑOS A PROCESAR (intersección población vs familiares)
# ============================================================
years_pobl = {int(c) for c in total_poblacion.columns if re.fullmatch(r"\d{4}", str(c))}
years_fam = {int(c) for c in year_cols}

years_to_process = sorted(years_pobl.intersection(years_fam))
print("Años a procesar:", years_to_process)

# ============================================================
# 5) FUNCIÓN PRINCIPAL
# ============================================================
def procesar_anio(anio: int):
    col = str(int(anio))

    numero_viviendas_familiares = (
        familiares_total[["key", "Nombre", col]]
        .rename(columns={col: "familiares_total"})
    )

    poblacion_anio = (
        total_poblacion[["key", col]]
        .rename(columns={col: "total_poblacion"})
    )

    datos = pd.merge(numero_viviendas_familiares, poblacion_anio, on="key", how="inner")

    datos = pd.merge(
        datos,
        municipios_cluster_ren[["key", "Nombre", "cluster"]],
        on="key",
        how="left",
        suffixes=("", "_cluster")
    )

    datos["Nombre"] = datos["Nombre_cluster"].fillna(datos["Nombre"])
    datos = datos.drop(columns=["Nombre_cluster"])

    datos["familiares_total"] = pd.to_numeric(datos["familiares_total"], errors="coerce")
    datos["total_poblacion"] = pd.to_numeric(datos["total_poblacion"], errors="coerce")

    # Indicador directo (no ratio)
    datos["Familiares"] = datos["familiares_total"].fillna(0)

    datos["type"] = 1  # más es mejor

    def calcular_atractividad_por_grupo(grupo: pd.DataFrame) -> pd.DataFrame:
        grupo = grupo.copy()
        media = grupo["Familiares"].mean()
        desviacion = grupo["Familiares"] - media
        max_dev = desviacion.abs().max()

        grupo["atractividad"] = (desviacion / max_dev) if (max_dev and max_dev != 0) else 0

        grupo["atractividad_0_100"] = grupo.apply(
            lambda row: 100 * ((row["atractividad"] + 1) / 2) if row["type"] == 1
            else 100 - (100 * ((row["atractividad"] + 1) / 2)),
            axis=1
        ).round(2)

        return grupo

    con_cluster = datos[datos["cluster"].notna()].copy()
    sin_cluster = datos[datos["cluster"].isna()].copy()

    con_cluster = con_cluster.groupby("cluster", group_keys=False).apply(calcular_atractividad_por_grupo)

    if not sin_cluster.empty:
        sin_cluster["atractividad_0_100"] = 0.0

    datos_final = pd.concat([con_cluster, sin_cluster], ignore_index=True)

    resultado = datos_final[["Nombre", "atractividad_0_100"]].rename(
        columns={"atractividad_0_100": "Familiares"}
    )

    output_path = f"data_interfaz/vivienda/normalizacion_final/{anio}/"
    os.makedirs(output_path, exist_ok=True)
    resultado.to_csv(f"{output_path}Familiares.csv", index=False)

    print(f"Archivo del {anio} exportado a {output_path}")

# ============================================================
# 6) PROCESAR (si solo quieres 2021, deja esto)
# ============================================================
#procesar_anio(2021)

# Si quisieras todos:
for y in years_to_process:
    procesar_anio(y)


Años a procesar: [2024]
Archivo del 2024 exportado a data_interfaz/vivienda/normalizacion_final/2024/


C:\Users\aitan\AppData\Local\Temp\ipykernel_17672\425992048.py:165: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  con_cluster = con_cluster.groupby("cluster", group_keys=False).apply(calcular_atractividad_por_grupo)


# *Datos idealista*

### **numBaños & precioVenta && numHabitaciones**

In [31]:
import pandas as pd
import re
import unicodedata
from pathlib import Path

# ========================
# CONFIGURACIÓN
# ========================
DIR_FILTRADOS = Path("data/vivienda/datos_filtrados")
CLUSTER_CSV = "external_data/municipios_con_cluster.csv"
OUTPUT_BASE = Path("data_interfaz/vivienda/normalizacion_final/2024")
OUTPUT_BASE.mkdir(parents=True, exist_ok=True)

# ========================
# HELPERS
# ========================
def canon_key(s):
    if pd.isna(s):
        return None
    s = str(s).lower().strip()
    s = unicodedata.normalize("NFKD", s)
    s = "".join(c for c in s if not unicodedata.combining(c))
    s = re.sub(r"[^a-z0-9]+", "", s)
    return s

def municipio_desde_filename(path: Path) -> str:
    return path.stem

# ========================
# ALIAS MANUALES (CASOS DIFÍCILES)
# ========================
ALIAS_MANUALES = {
    "alamedavalle": "Alameda del Valle",
    "aldeafresno": "Aldea del Fresno",
    "becerrilsierra": "Becerril de la Sierra",
    "belmontetajo": "Belmonte de Tajo",
    "berzosalozoya": "Berzosa del Lozoya",
    "breatajo": "Brea de Tajo",
    "cabanillassierra": "Cabanillas de la Sierra",
    "cadalsovidrios": "Cadalso de los Vidrios",
    "camarmaesteruelas": "Camarma de Esteruelas",
    "cerverabuitrago": "Cervera de Buitrago",
    "colmenararroyo": "Colmenar del Arroyo",
    "colmenaroreja": "Colmenar de Oreja",
    "cubassagra": "Cubas de la Sagra",
    "elalamo": "Alamo (El)",
    "madarcosreferences":"Madarcos",
    "montejosierrareferences":"Montejo de la Sierra",
    "navarredondasomereferences":"Navarredonda y San Mamés",
    "puentesviejassomereferences":"Puentes Viejas",
    "elatazar": "Atazar (El)",
    "elberrueco": "Berrueco (El)",
    "elboalo": "Boalo (El)",
    "elmolar": "Molar (El)",
    "elvellon": "Vellón (El)",
    "fresnedillasoliva": "Fresnedillas de la Oliva",
    "fresnotorote": "Fresno de Torote",
    "fuentesaz": "Fuente el Saz de Jarama",
    "fuentiduenatajo": "Fuentidueña de Tajo",
    "gargantamontes": "Garganta de los Montes",
    "gargantillalozoya": "Gargantilla del Lozoya y Pinilla de Buitrago",
    "guadalixsierra": "Guadalix de la Sierra",
    "horcajosierra": "Horcajo de la Sierra-Aoslos",
    "horcajuelosierra": "Horcajuelo de la Sierra",
    "hoyomanzanares": "Hoyo de Manzanares",
    "humanesmadrid": "Humanes de Madrid",
    "lacabrera": "Cabrera (La)",
    "lahiruela": "Hiruela (La)",
    "lasernamonte": "Serna del Monte (La)",
    "lasrozas": "Rozas de Madrid (Las)",
    "losmolinos": "Molinos (Los)",
    "lossantoshumosa": "Santos de la Humosa (Los)",
    "loyozuela": "Lozoyuela-Navas-Sieteiglesias",
    "manzanaresreal": "Manzanares el Real",
    "mejoradacampo": "Mejorada del Campo",
    "mirafloressierra": "Miraflores de la Sierra",
    "moralejaenmedio": "Moraleja de Enmedio",
    "moratatajuna": "Morata de Tajuña",
    "navasrey": "Navas del Rey",
    "olmedafuentes": "Olmeda de las Fuentes",
    "pelayospresa": "Pelayos de la Presa",
    "peralestajuna": "Perales de Tajuña",
    "pezuelatorres": "Pezuela de las Torres",
    "pinillavalle": "Pinilla del Valle",
    "pozueloalarcon": "Pozuelo de Alarcón",
    "pozuelorey": "Pozuelo del Rey",
    "pradenarincon": "Prádena del Rincón",
    "pueblasierra": "Puebla de la Sierra",
    "robledillojara": "Robledillo de la Jara",
    "robledochavela": "Robledo de Chavela",
    "rozaspuertoreal": "Rozas de Puerto Real",
    "sanagustinguadalix": "San Agustín del Guadalix",
    "sanfernandohenares": "San Fernando de Henares",
    "sanlorenzoescorial": "San Lorenzo de El Escorial",
    "sanmartinvaldeiglesias": "San Martín de Valdeiglesias",
    "sanmartinvega": "San Martín de la Vega",
    "sansebastianreyes": "San Sebastián de los Reyes",
    "santamariaalameda": "Santa María de la Alameda",
    "serranillosvalle": "Serranillos del Valle",
    "sevillanueva": "Sevilla la Nueva",
    "sotoreal": "Soto del Real",
    "talamancajarama": "Talamanca de Jarama",
    "torrejonardoz": "Torrejón de Ardoz",
    "torrejoncalzada": "Torrejón de la Calzada",
    "torrejonvelasco": "Torrejón de Velasco",
    "torremochajarama": "Torremocha de Jarama",
    "torresalameda": "Torres de la Alameda",
    "valdeolmosalapardo": "Valdeolmos-Alalpardo",
    "valdetorresjarama": "Valdetorres de Jarama",
    "valverdealcala": "Valverde de Alcalá",
    "velillasanantonio": "Velilla de San Antonio",
    "villamanriquetajo": "Villamanrique de Tajo",
    "villanuevacanada": "Villanueva de la Cañada",
    "villanuevapardillo": "Villanueva del Pardillo",
    "villanuevaperales": "Villanueva de Perales",
    "villaprado": "Villa del Prado",
    "villarejosalvanes": "Villarejo de Salvanés",
    "villarolmo": "Villar del Olmo",
    "villaviciosaodon": "Villaviciosa de Odón",
    "villaviejalozoya": "Villavieja del Lozoya"
}

# ========================
# 1) CARGA Y MEDIA
# ========================
def cargar_datos_y_agrupar():
    frames = []
    files = list(DIR_FILTRADOS.glob("*.csv"))
    print(f"📂 Encontrados {len(files)} CSV de vivienda")

    for csv in files:
        df = pd.read_csv(csv)
        df["key"] = canon_key(municipio_desde_filename(csv))
        frames.append(df)

    df_all = pd.concat(frames, ignore_index=True)

    for c in ["precio", "habitaciones", "banos"]:
        df_all[c] = pd.to_numeric(df_all[c], errors="coerce")

    return (
        df_all
        .groupby("key", as_index=False)[["precio", "habitaciones", "banos"]]
        .mean()
        .round(2)
    )

# ========================
# 2) MATCH MUNICIPIOS
# ========================
def emparejar_municipios(df_keys, municipios_cluster):
    municipios_cluster = municipios_cluster.copy()
    municipios_cluster["key_cluster"] = municipios_cluster["Nombre"].apply(canon_key)

    rows = []

    for key in df_keys:
        if key in ALIAS_MANUALES:
            match = municipios_cluster[
                municipios_cluster["Nombre"] == ALIAS_MANUALES[key]
            ]
        else:
            match = municipios_cluster[
                municipios_cluster["key_cluster"].str.contains(key, na=False)
            ]

        if not match.empty:
            row = match.iloc[0]
            rows.append({
                "key": key,
                "Nombre": row["Nombre"],
                "cluster": row["Cluster"]
            })

    return pd.DataFrame(rows)

# ========================
# 3) ATRACTIVIDAD
# ========================
def calcular_atractividad(df, variable, tipo):
    def normalizar(g):
        m = g[variable].mean()
        d = g[variable] - m
        md = d.abs().max()
        g["score"] = (d / md) if md and md != 0 else 0
        g["score_0100"] = g["score"].apply(
            lambda x: 100 * ((x + 1) / 2)
            if tipo == 1 else 100 - (100 * ((x + 1) / 2))
        ).round(2)
        return g

    return df.groupby("cluster", group_keys=False).apply(normalizar)

# ========================
# 4) PROCESAR VARIABLE
# ========================
def procesar_variable(df, municipios_cluster, variable, tipo):
    mapping = emparejar_municipios(df["key"].unique(), municipios_cluster)
     # ⬇️ AÑADE ESTO
    keys_en_datos = set(df["key"].unique())
    keys_emparejados = set(mapping["key"].unique())
    no_emparejados = sorted(keys_en_datos - keys_emparejados)

    if no_emparejados:
        print(f"❌ No emparejados: {len(no_emparejados)}")
        print("🔍 Claves sin emparejar:")
        for k in no_emparejados:
            print(f"  - {k}")
    else:
        print("✅ Todos los municipios emparejados correctamente")

    df = pd.merge(df, mapping, on="key", how="inner")

    resultado = calcular_atractividad(df, variable, tipo)

    salida = (
        resultado[["Nombre", "score_0100"]]
        .rename(columns={"score_0100": variable.capitalize()})
        .sort_values("Nombre")
    )

    salida.to_csv(OUTPUT_BASE / f"{variable.capitalize()}.csv", index=False)
    print(f"✅ {variable.capitalize()}.csv generado")

# ========================
# MAIN
# ========================
if __name__ == "__main__":
    municipios_cluster = pd.read_csv(CLUSTER_CSV)
    datos = cargar_datos_y_agrupar()

    procesar_variable(datos, municipios_cluster, "precio", tipo=0)
    procesar_variable(datos, municipios_cluster, "habitaciones", tipo=1)
    procesar_variable(datos, municipios_cluster, "banos", tipo=1)

    print("🎉 TODO COMPLETADO")


📂 Encontrados 178 CSV de vivienda
✅ Todos los municipios emparejados correctamente
✅ Precio.csv generado
✅ Todos los municipios emparejados correctamente
✅ Habitaciones.csv generado
✅ Todos los municipios emparejados correctamente
✅ Banos.csv generado
🎉 TODO COMPLETADO


C:\Users\aitan\AppData\Local\Temp\ipykernel_17672\2408648550.py:192: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  return df.groupby("cluster", group_keys=False).apply(normalizar)
C:\Users\aitan\AppData\Local\Temp\ipykernel_17672\2408648550.py:192: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  return df.groupby("cluster", group_keys=False).apply(normalizar)
C:\Users\aitan\AppData\Local\Temp\ipykernel_17672\